In [2]:
## we define the product type here, Landsat 5:"CU_LT05.001", Landsat 7 :"CU_LE07.001", Landsat 8 :"CU_LC08.001"
prods = ["CU_LT05.001"]
layers = [(prods[0], "PIXELQA"), (prods[0], "SRB1"), (prods[0], "SRB2"), (prods[0], "SRB3"), (prods[0], "SRB4"), (prods[0], "SRB5"), (prods[0], "SRB7"), (prods[0], "SRCLOUDQA")]
prodLayer = []
for l in layers:
    prodLayer.append({
        "layer": l[1],
        "product": l[0]
    })
prodLayer

[{'layer': 'PIXELQA', 'product': 'CU_LT05.001'},
 {'layer': 'SRB1', 'product': 'CU_LT05.001'},
 {'layer': 'SRB2', 'product': 'CU_LT05.001'},
 {'layer': 'SRB3', 'product': 'CU_LT05.001'},
 {'layer': 'SRB4', 'product': 'CU_LT05.001'},
 {'layer': 'SRB5', 'product': 'CU_LT05.001'},
 {'layer': 'SRB7', 'product': 'CU_LT05.001'},
 {'layer': 'SRCLOUDQA', 'product': 'CU_LT05.001'}]

In [3]:
## establish credendtial for the api
import requests as r
import getpass, pprint, time, os, cgi, json
import geopandas as gpd

## Enter Earth Data login Credentials
username = "ZekunLin"#getpass.getpass('Earthdata Username:')
password = "P6Fn8T47"#getpass.getpass('Earthdata Password:')

api = 'https://lpdaacsvc.cr.usgs.gov/appeears/api/'  # Set the AρρEEARS API to a variable

token_response = r.post('{}login'.format(api), auth=(username, password)).json() # Insert API URL, call login service, provide credentials & return json
del username, password                                                           # Remove user and password information
token_response 

{'token_type': 'Bearer',
 'token': 'NFLiWmOmZWaO7zcS6NI9naRyeO2qHCYALGgeYHCPrM3Gr6zlXg1F_Yec3p9WgqiRst1HP9q7uy3coQ1EbL5ivg',
 'expiration': '2020-10-14T15:39:49Z'}

In [4]:
# look up the available projections 
projections = r.get('{}spatial/proj'.format(api)).json()  # Call to spatial API, return projs as json
projections
projs = {}
for p in projections:
    projs[p["Name"]] = p
list(projs.keys())



['native',
 'geographic',
 'sinu_modis',
 'albers_weld_alaska',
 'albers_weld_conus',
 'albers_ard_alaska',
 'albers_ard_conus',
 'albers_ard_hawaii',
 'easegrid_2_global',
 'easegrid_2_north',
 'laea_sphere_19']

In [21]:
## submit the orders by tiles
import os
import glob
import time

tile_list = glob.glob("D:/Zekun/Landsat_ARD/Landsat_ARD/ARD_Uni_shapefiles/*.shp")      # where the 75 tiles stored
short_name = [os.path.basename(x) for x in glob.glob("D:/Zekun/Landsat_ARD/Landsat_ARD/ARD_Uni_shapefiles/*.shp")]  # get the tile names

period1 = ["01-01-1984", "12-31-1999"]                  # due to limits of APPEEARS' data process capability we need to download data by 2 periods
period2 = ["01-01-2000", "12-31-2013"]
periods = {"1" : period1, "2" : period2}

for p in ["1", "2"]:

    for i in range(len(tile_list)):

        nps = gpd.read_file(tile_list[i])
        nps = nps.to_crs("EPSG:4326")

        nps_gc = nps.to_json()
        nps_gc = json.loads(nps_gc)

        task_name = "p" + p + "_" + short_name[i][0:7]        # task name = period + tile name
        task_type = "area"                                    # we are using polygon instead of points to extract data
        proj = projs["geographic"]["Name"]                    # it MUST BE geographic projection make this api work
        outFormat = ["geotiff"]
        startDate = periods[p][0]
        endDate = periods[p][1]
        recurring = False

        task = {                                # build a json type of geo area to indicate ROI
            "task_type": task_type,             # area sample task
            "task_name": task_name,
            "params": {
                "dates": [
                {
                    "startDate": startDate,
                    "endDate": endDate
                }],
                "layers": prodLayer,
                "output": {
                    "format": {
                        "type": outFormat[0]    # geotiff format
                    },
                    "projection": proj
                },
                "geo": nps_gc
                }
        }


        token = token_response['token']                      # Save login token to a variable
        head = {'Authorization': 'Bearer {}'.format(token)}  # Create a header to store token information, needed to submit a request

        task_response = r.post("{}task".format(api), json = task, headers = head).json()
        print(task_response)
        
        task_id = task_response['task_id']                                               # Set task id from request submission
        status_response = r.get('{}status/{}'.format(api, task_id), headers=head).json() # Call status service with specific task ID & user credentials
        print(status_response['status'])

        write_task_id = task_id + " : " + task_name + ":" + status_response['status']
        t = open(os.path.join("D:/Zekun/Landsat_ARD/Landsat_ARD/se_ard/", "LT5_task_id_by_tile.txt"), 'a')
        t.write(write_task_id +"\n")
        t.close()

        print('orders have been successfully submitted.')

{'task_id': 'a3507b50-bd6a-4c61-8f84-b87adeca7401', 'status': 'pending'}
pending
{'task_id': 'feffeca6-a8a2-4965-a455-a06fb9929ca4', 'status': 'pending'}
pending
{'task_id': '968326fa-7a0d-48f9-8682-aca4af6e35a1', 'status': 'pending'}
pending
{'task_id': 'e29a2e8c-e62b-449e-af2f-ccd1e0159113', 'status': 'pending'}
pending
{'task_id': '8d10d518-e4cf-4f08-b12b-ce364e4589e6', 'status': 'pending'}
pending
{'task_id': '2a81ed00-ddaf-49f9-88d1-92999ff80ad7', 'status': 'pending'}
pending
{'task_id': 'ebd6823c-755d-416e-b410-607fbc410169', 'status': 'pending'}
pending
{'task_id': '0890126c-5ac1-46b1-89b3-1caa904dcf07', 'status': 'pending'}
pending
{'task_id': 'cde084e3-ddb2-43a4-8459-15c0f6e8e6cb', 'status': 'pending'}
pending
{'task_id': 'b9788e0e-beac-4760-a714-8d3f69297aff', 'status': 'pending'}
pending
{'task_id': '965264ab-d337-4d51-a91f-85f9849a7bf2', 'status': 'pending'}
pending
{'task_id': 'b2906c1e-8f45-4e03-91e3-c51a84042c5b', 'status': 'pending'}
pending
{'task_id': '5c9d7515-6fbf-4

KeyError: 'task_id'

In [20]:
## we start to download the data if they have been processed
import requests as r
import getpass, pprint, time, os, cgi, json

token = token_response['token']                                                     # retrieve the existed orders in the Appeears
response = r.get('https://lpdaacsvc.cr.usgs.gov/appeears/api/task', 
    headers={'Authorization': 'Bearer {0}'.format(token)})
task_response = response.json()

t = open("D:/Zekun/Landsat_ARD/Landsat_ARD/se_ard/LT5_task_id_by_tile.txt", "r")    # read the list of orders
task_id_list = t.read().split('\n')

for x in range(len(task_id_list)):
    submited_id = task_id_list[x][0:36]
    for y in range(len(task_response)):
        if task_response[y]['task_id'] == submited_id:
            if task_response[y]['status'] == 'done':
                download_this_name = task_response[y]['task_name']
                download_this_id = task_response[y]['task_id']
                bundle = r.get('{}bundle/{}'.format(api, download_this_id)).json() # Call API and return bundle contents for the task_id as json
                #print(bundle['files'])
                files = {}                                                         # Create empty dictionary
                for f in bundle['files']: files[f['file_id']] = f['file_name']    # Fill dictionary with file_id as keys and file_name as values

                destDir = os.path.join("D:\Zekun\Landsat_ARD\Landsat_ARD\se_ard", prods[0], download_this_name)
                if not os.path.exists(destDir) : os.makedirs(destDir)

                for f in files:
                    dl = r.get('{}bundle/{}/{}'.format(api, download_this_id, f), stream = True)                      # Get a stream to the bundle file
                    filename = os.path.basename(cgi.parse_header(dl.headers['Content-Disposition'])[1]['filename'])   # Parse the name from Content-Disposition header
                    filepath = os.path.join(destDir, filename)                                                        # Create output file path
                    with open(filepath, 'wb') as f:                                                                   # Write file to dest dir
                        for data in dl.iter_content(chunk_size=8192): f.write(data)
                print('Downloaded files can be found at: {}'.format(destDir))

    else: pass




Downloaded files can be found at: D:\Zekun\Landsat_ARD\Landsat_ARD\se_ard\CU_LT05.001\test_1


In [11]:
token = token_response['token']                                                     # retrieve the existed orders in the Appeears
response = r.get('https://lpdaacsvc.cr.usgs.gov/appeears/api/task', 
    headers={'Authorization': 'Bearer {0}'.format(token)})
task_response = response.json()

download_this_name = task_response[1]['task_name']
os.path.join("D:\Zekun\Landsat_ARD\Landsat_ARD\se_ard", prods[0], download_this_name)

'D:\\Zekun\\Landsat_ARD\\Landsat_ARD\\se_ard\\CU_LT05.001\\p1_h22_v10'

In [97]:

bundle = r.get('{}bundle/{}'.format(api,task_id)).json()  # Call API and return bundle contents for the task_id as json


files = {}                                                       # Create empty dictionary
for f in bundle['files']: files[f['file_id']] = f['file_name']   # Fill dictionary with file_id as keys and file_name as values


destDir = os.path.join("D:\Zekun\Landsat_ARD\Landsat_ARD\se_ard\CU_LT05.001", task_name)
if not os.path.exists(destDir) : os.makedirs(destDir)

for f in files:
    dl = r.get('{}bundle/{}/{}'.format(api, task_id, f), stream=True)           # Get a stream to the bundle file
    filename = os.path.basename(cgi.parse_header(dl.headers['Content-Disposition'])[1]['filename'])  # Parse the name from Content-Disposition header 
    filepath = os.path.join(destDir, filename)                                                       # Create output file path
    with open(filepath, 'wb') as f:                                                                  # Write file to dest dir
        for data in dl.iter_content(chunk_size=8192): f.write(data) 
print('Downloaded files can be found at: {}'.format(destDir)) 

' bundle = r.get(\'{}bundle/{}\'.format(api,task_id)).json()  # Call API and return bundle contents for the task_id as json\n\n\nfiles = {}                                                       # Create empty dictionary\nfor f in bundle[\'files\']: files[f[\'file_id\']] = f[\'file_name\']   # Fill dictionary with file_id as keys and file_name as values\n\n\ndestDir = os.path.join("D:\\Zekun\\Landsat_ARD\\Landsat_ARD\\se_ard\\CU_LT05.001", task_name)\nif not os.path.exists(destDir) : os.makedirs(destDir)\n\nfor f in files:\n    dl = r.get(\'{}bundle/{}/{}\'.format(api, task_id, f), stream=True)           # Get a stream to the bundle file\n    filename = os.path.basename(cgi.parse_header(dl.headers[\'Content-Disposition\'])[1][\'filename\'])  # Parse the name from Content-Disposition header \n    filepath = os.path.join(destDir, filename)                                                       # Create output file path\n    with open(filepath, \'wb\') as f:                                 

In [89]:
t = open("D:/Zekun/Landsat_ARD/Landsat_ARD/se_ard/LT5_task_id_by_tile.txt", "r")
task_id_list = t.read().split('\n')
task_id = task_id_list[0][0:36]
status_response = r.get('{}status/{}'.format(api, task_id), headers=head).json() # Call status service with specific task ID & user credentials
status_response['status']

'pending'